## Notebook to display boreholes on a map and to create the gpkg database from an existing sql3database and shapefile data

Create gpkg from shp 

In [1]:
import geopandas as gpd
import pandas as pd
import folium
import sqlite3

read shp file (as a geodataframe)

In [2]:
dat_shp = gpd.read_file("/home/gfa/GSDMA2019/playground/pilote/pilote/Pilote_v7.0.shp")


In [3]:
cd /home/gfa/GSDMA2019/playground/pilote/pilote

/home/gfa/GSDMA2019/playground/pilote/pilote


In [4]:
ls

data.gpkg                                          Pilote_v7.0.prj
dat.gpkg                                           Pilote_v7.0.sbn
Decoupe_tube_2019.txt                              Pilote_v7.0.sbx
final.db                                           Pilote_v7.0.shp
Notes_terrain_Memoris_2019-10-24_RecoupeTubes.pdf  Pilote_v7.0.shx
Pilote_v7.0.dbf                                    project_database.db


Read GPKG as a GeoDataFrame

Write a geojson from gdf

In [5]:
data_from_gpkg.to_file("data.geojson", driver='GeoJSON')

NameError: name 'data_from_gpkg' is not defined

## CRS

EPSG 31370 (Lambert 72)

In [6]:
dat_84 = dat_shp.to_crs(epsg=4326)

/home/gfa/.local/share/virtualenvs/GSDMA2019-F9ES6BKJ/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


### Create GPKG from gdf

In [7]:
dat_84.to_file("dat.gpkg", driver="GPKG")

### List of Tables

In [8]:
list_tables = ['Components', 'Intervals', 'Boreholes','Lexicon']

### Copy Tables

In [11]:
db_copy_sqlite3_tables('project_database.db','dat.gpkg',list_tables)

TypeError: unsupported format string passed to DatabaseError.__format__

## Folium 

In [ ]:
dat_gdf = gpd.read_file("dat.gpkg")


In [ ]:
dat_84.to_file("output.json", driver="GeoJSON")

In [ ]:
ice_map = folium.Map(location=[50.455, 3.94],
                    zoom_start=10)
folium.GeoJson(
    'output.json').add_to(ice_map)
ice_map

save in a file

In [ ]:
ice_map.save('index.html')

## Access SQlite

In [ ]:
conn = sqlite3.connect('/home/gfa/test/GSDMA2019-develop/project_database.db')

In [ ]:
conn

In [ ]:
cursor = conn.cursor()

Check which tables are in db

In [ ]:
for table in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(table)

Access the data 

### Copy tables of a SQL3 database to another database

In [10]:
def db_copy_sqlite3_tables(db_in, db_out, tables):
    """
    Copy tables from one database to another
    :param db_in: full filename of the origin of tables 
    :type db_in: string
    :param db_out: full filename of the destination of tables
    :type db_out: string
    :param tables: list of tables to copy
    :type tables: list
    :return: status, 0: OK, 1: failure
    :rtype: int
    """
    import sqlite3
    import pandas as pd
    
    status = 0
    try:
        conn_in = sqlite3.connect(db_in)
        conn_out = sqlite3.connect(db_out)
        for i in tables:
            df = pd.read_sql_query("SELECT * from {table:s}".format(table=i), conn_in)
            df.to_sql("{table:s}".format(table=i),conn_out, if_exists='replace', index = False)
        conn_in.close()
        conn_out.close()
    except Exception as e:
        print('Error copying tables:\n {error:s}'.format(error=e))
        status = 1
    return status    

NameError: name 'cd' is not defined

In [15]:
db_copy_sqlite3_tables('final.db','data.gpkg',list_tables)

TypeError: unsupported format string passed to DatabaseError.__format__

In [ ]:
db_copy_sqlite3_tables('project_database.db','dat.gpkg',list_tables)

In [ ]:
dat_gdf = gpd.read_file("dat.gpkg")

Add column with the name of the borehole

In [ ]:
dat_gdf['id'] = 'F'+data_from_gpkg['Ref']